## BMI for the bionic arm

As a proof of concept we want to show that our bionic arm can replicate the actions performed by the monkey during a trial.

### Import

In [1]:
import paho.mqtt.client as paho
import numpy as np
from time import sleep
from datetime import datetime
import json
from tensorflow.keras.models import load_model

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Parameters

In [2]:
clientID = 'bmi'
broker = 'localhost'
port = 1883
topic = 'brain_activity'

## 📲 Initialize the XBee radio module

Specify which serial port the transmitting Xbee radio module is connected to:

In [3]:
XBEE_PORT = 'COM5'

> ℹ️ **NOTE:** On Linux, the port name of the XBee radio module (transmitter) will be
   - `/dev/ttyACMX` when using e.g. Arduino + shield;
   - `/dev/ttyUSBX` when using a dedicated USB interface.

Import the XBee modules and create an XBee device object with the serial port we defined earlier and a baud rate of 115200:

In [4]:
from digi.xbee.devices import XBeeDevice, RemoteXBeeDevice, XBee64BitAddress

xbee = XBeeDevice(XBEE_PORT, 115200)

Open the connection to the XBee device:

In [5]:
xbee.open()

SerialException: could not open port 'COM5': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

Print the node name and MAC address of this XBee device so we know we have the correct one:

In [7]:
print("Node ID: %s \t MAC: %s" % (xbee.get_node_id(), xbee.get_64bit_addr()) )

Node ID: TRANSMITTER 	 MAC: 0013A20041CB9200


Try to find the aggregator node on the network so we can send data to it later:

In [9]:
#xbee_network = xbee.get_network()

#aggregator_xbee = xbee_network.get_device_by_node_id("AGGREGATOR")

AGGREGATOR_MAC = "0013A20041C14D4A"
aggregator_xbee = RemoteXBeeDevice(xbee, XBee64BitAddress.from_hex_string(AGGREGATOR_MAC))

#if aggregator_xbee is None:
#    raise ValueError('Aggregator node not found!')

### Script

In [3]:
# Initialize the publisher
client = paho.Client(clientID, False)

# Initialize buffer
global buffer
buffer = None

# Initialize model
shape_model = load_model('../results/online_decoding/shape_decoder_10.h5')
size_model = load_model('../results/online_decoding/size_decoder_10.h5')
state_model = load_model('../results/online_decoding/state_decoder_10.h5')

def myOnMessageReceived(paho_mqtt, userdata, msg):
    global buffer
    if msg.topic == topic:
        message = json.loads(msg.payload)
        if message['status'] == 'starting_simulation':
            print('Simulation started!')
            buffer = np.empty((message['shape'],))
            
        elif message['status'] == 'running_simulation':
            buffer = np.vstack((buffer, message['value']['measurements']))
            print(buffer.shape)
            next_state = state_model.predict(buffer.T)
            if next_state in ['Cue', 'Planning', 'Moviment']:
                shape = shape_model.predict(buffer.T)
                size = size_model.predict(buffer.T)
            else:
                shape = None
                size = None
            print(message['value']['index'], '# ', buffer.shape, phase, shape, size)
        
        elif message['status'] == 'ending_simulation':
            print('Simulation terminated')
            buffer = None
            
            
        
client.on_message = myOnMessageReceived

In [4]:
client.connect(broker, port)
client.loop_start()
client.subscribe(topic, 2)
print('Waiting for simulation...')
while True:
    try:
        sleep(1)
        
    except KeyboardInterrupt:
        print('Simulation interrupted')
        client.unsubscribe(topic)
        client.loop_stop()
        client.disconnect()
        sys.exit(0)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "D:\miniconda3\envs\thesis\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "D:\miniconda3\envs\thesis\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "D:\miniconda3\envs\thesis\lib\site-packages\paho\mqtt\client.py", line 3452, in _thread_main
    self.loop_forever(retry_first_connection=True)
  File "D:\miniconda3\envs\thesis\lib\site-packages\paho\mqtt\client.py", line 1779, in loop_forever
    rc = self.loop(timeout, max_packets)
  File "D:\miniconda3\envs\thesis\lib\site-packages\paho\mqtt\client.py", line 1181, in loop
    rc = self.loop_read(max_packets)
  File "D:\miniconda3\envs\thesis\lib\site-packages\paho\mqtt\client.py", line 1572, in loop_read
    rc = self._packet_read()
  File "D:\miniconda3\envs\thesis\lib\site-packages\paho\mqtt\client.py", line 2310, in _packet_read
    rc = self._packet_handle()
  File "D:\miniconda3\envs\thesis\lib\

Waiting for simulation...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Simulation interrupted
Traceback (most recent call last):
  File "<ipython-input-4-de8a415a5d19>", line 7, in <module>
    sleep(1)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\miniconda3\envs\thesis\lib\site-packages\IPython\core\interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-de8a415a5d19>", line 14, in <module>
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "D:\miniconda3\envs\thesis\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "D:\miniconda3\envs\thesis\lib\site-packages\IPython\core\ultratb.py", line 248, in wrapped
    return f(*args, **kwargs)
  File "D:\miniconda3\envs\thesis\lib\site-packages\IPython\core\ultratb.p

TypeError: object of type 'NoneType' has no len()

## 📡 Transmit 802.15.4 packet with XBee radio module

Send the payload that we defined earlier to the aggregator/receiving XBee:

In [12]:
tx_status = xbee.send_data(aggregator_xbee, payload)

print(tx_status)


{<DictKeys.HEADER_BYTE: 'header'>: 126, <DictKeys.LENGTH: 'length'>: 3, <DictKeys.FRAME_SPEC_DATA: 'fr_spec_data'>: {<DictKeys.FRAME_TYPE: 'fr_type'>: <ApiFrameType.TX_STATUS: (137, 'TX (Transmit) Status')>, <DictKeys.FRAME_ID: 'fr_id'>: 9, <DictKeys.API_DATA: 'api_spec_data'>: {<DictKeys.TS_STATUS: 'ts_status'>: <TransmitStatus.SUCCESS: (0, 'Success.')>}}, <DictKeys.CHECKSUM: 'checksum'>: 109}


In [ ]:
# 5 fingers + wrist + elbow + approaching
actuator_dict = {
    "ball":[175,175,175,175,175, 80, 255, X],
}

payload = bytearray(actuator_dict[pred]+)

